# Tic Tac Toe

You only need to write your code in Step 5. Please read through and understand all other codes before writing your own code. 

In [46]:
from collections import namedtuple, Counter, defaultdict
import random
import math

### Cache in Python

The @cache decorator in Python is used to store previously computed results of a function, so that if the function is called again with the same arguments, the stored result is returned instantly, instead of recomputing it.

In [47]:
import functools 
cache = functools.lru_cache(10**6)

Fibonacci without cache:

In [48]:
%%time

def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(35))  # Takes a long time


9227465
CPU times: user 945 ms, sys: 6.9 ms, total: 952 ms
Wall time: 957 ms


Fibonacci with cache:

In [49]:
%%time

@cache 
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(35))  # Takes a much short time

9227465
CPU times: user 107 μs, sys: 45 μs, total: 152 μs
Wall time: 203 μs


## Step 1. Define Class of Tic-Tac-Toe

## Step 1.1. Base Class: Game

We start with defining the abstract class `Game`, for turn-taking *n*-player games. We rely on, but do not define yet, the concept of a `state` of the game; we'll see later how individual games define states. For now, all we require is that a state has a `state.player` attribute, which gives the name of the player whose turn it is. ("Name" will be something like `'X'` or `'O'` for tic-tac-toe.) 

In [6]:
class Game:
    """A game is similar to a problem, but it has a terminal test instead of 
    a goal test, and a utility for each terminal state. To create a game, 
    subclass this class and implement `actions`, `result`, `is_terminal`, 
    and `utility`. You will also need to set the .initial attribute to the 
    initial state; this can be done in the constructor."""

    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from making an action/move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)
    
    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError
        

### Step 1.2. Define a Board class

States in tic-tac-toe (and other games) will be represented as a `Board`, which is a subclass of `defaultdict` that in general will consist of `{(x, y): contents}` pairs, for example `{(0, 0): 'X', (1, 1): 'O'}` might be the state of the board after two moves. Besides the contents of squares, a board also has some attributes:

- `.player` to name the player whose move it is; 
- `.width` and `.height` to give the size of the board (both 3 in tic-tac-toe, but other numbers in related games);
- possibly other attributes like utility, as specified by keywords. 

As a `defaultdict`, the `Board` class has a `__missing__` method, which returns `empty` for squares that have no been assigned but are within the `width` × `height` boundaries, or `#` otherwise. 

In [7]:

"""
Two purposes of creating our own customized dictionary:
1. to handle attributes. Each board is a dictionary {position tuple: player}. It has player and utility attributes. 
2. to handle missing key. We do not initialize all 9 positions as keys at first, but treat all of them as missing keys. 
We assign key at one position after one player moves at this position. 
3. to hash board for using cache decorator 
"""

class Board(defaultdict):
    """A board has the player to move, a cached utility value, 
    and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""

    
    def __init__(self, width=3, height=3, player=None, **kwds):
        self.__dict__.update(width=width, height=height, player=player, **kwds)
        """ 
        width and height are the size of the board. 
        player is the player of current board. player will make a move on current board (self). 
        A new board will be created after player makes a move. 
        
        width, height, player, etc are all class attributes. 
        Class attributes of this self-defined dictionary are not key-value pairs in .items. 
        
        The line above is equivalent to the code block below. It updates the class attributes 
        width, height, player (X or O), and more like utility (1, -1, or 0) via **kwds.
        """
        # self.width = width
        # self.height = height
        # self.palyer = palyer 
        # for key, value in kwds.items():
        #     setattr(self, key, value) # Do not use self.key = value since it always sets an attribute named "key" rather than using the variable key as the attribute name.
        
    def make_new_board(self, changes: dict, **kwds) -> 'Board':
        "Given a dict of {(x, y): contents} changes, return a new Board with the changes."
        # Create and return a new board as state after every change. 
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self) # copy items of current board
        board.update(changes) # create a new item of the move (position: player) in the new board
        return board

    def __missing__(self, loc):
        """
        loc is key (column, row) 
        This function is called when there is no key "loc" in current dictionary. 
        If loc is within the boundary, the value is the corresponding integer. 
        Otherwise, show out of boundary symbol #. 
        """
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return str(y*self.width+x+1)
        else:
            return "#" 
            
    # We need hash board for using cache decorator 
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.player)
    
    def __repr__(self):
        """
        Define customized output for print the board.
        Loop over pairs from (0,0) to (3,3) and show X/O if there is a move at it.
        Otherwise, call __missing__ to show an integer instead. So, initially, the board is
            1 2 3
            4 5 6
            7 8 9
        """
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'

### Step 1.3. Define game class for Tic Tac Toe

We have the notion of an abstract game, we have some search functions; now it is time to define a real game; a simple one, tic-tac-toe. Moves are `(x, y)` pairs denoting squares, where `(0, 0)` is the top left, and `(2, 2)` is the bottom right (on a board of size `height=width=3`).

In following class, we define 

- state is board object and action is a coordinate on the board
- initilization of the game, which provides initial state
- action(s)
- result(s, a)
- utility(s, p)
- is_terminal(s)
- terminal_test(s, p, a)


In [8]:
class TicTacToe(Game):
    """Play TicTacToe on an `height` by `width` board, needing `k` in a row to win.
    'X' plays first against 'O'."""
    # state of the game is board
    # board is a dictionary with format {(i,j): player}
    # board has attribute current player and utility 

    def __init__(self, height=3, width=3):
        # use set comprehension to initialize the set below 
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, player='X', utility=0) # utility via **kwds in Board

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        # actions is a list of unoccupied positions (i,j). 
        # set(dict) returns a set of keys of the dict. Here, set(board) returns all current occupied positions. 
        return self.squares - set(board)

    def result(self, board, action):
        """Place a marker for current player on square."""
        # action is a tuple of position 
        # apply one action on current board by updating the dictinoary of the board with square.
        # return a new created board. 
        player = board.player
        board = board.make_new_board({action: player}, player=('O' if player == 'X' else 'X'))
        
        win = self.terminal_test(board, player, action)
        board.utility = (0 if not win else +1 if player == 'X' else -1) # update the utility if win 
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def terminal_test(self, board, player, square):
        """True if player has 3 pieces in a line through square."""
        def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
        return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= 3
                   for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

## Step 2. Define Players

We need an interface for players. I'll represent a player as a `callable` that will be passed two arguments: `(game, state)` and will return a `move`.
The function `player` creates a player out of a search algorithm, but you can create your own players as functions, as is done with `random_player` below.

There three players:

1. random player
2. human player
3. AI player

Each player receives game object and current state, and returns an action. 

In [9]:
# A player plays randomly
def random_player(game, state): return random.choice(list(game.actions(state)))

# A human player who types the action at each turn
def human_player(game, state):
    while True:
        pos_raw = input("Please input a new position 1-9:")
        pos = [int(i) for i in pos_raw.split()]
        if len(pos) != 1:
            print("Please input only one number!")
        else:
            pos = pos[0]
            if pos < 1 or pos > 9:
                print("Please input a number between 1 and 9!")
            else:
                # top left is 0 0 and bottom right is 2 2
                x,y = (pos-1)%3,(pos-1)//3
                if (x,y) not in game.actions(state):
                    print("Please input an available position!")
                else:
                    return (x,y)
                
# An AI player 
# Call and return a search algorithm, which will be defined later on. 
def AI_player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    # Only return the move, not the value of utility. 
    return lambda game, state: search_algorithm(game, state)[1]

## Step 3. Define the game loop 

We define `play_game`, which takes a game and a dictionary of  `{player_name: strategy_function}` pairs, and plays out the game, on each turn checking `state.player` to see whose turn it is, and then getting the strategy function for that player and applying it to the game and the state to get a move.

For example, we pass following two players in strategies to the game loop:

- X: random player
- O: human player

In [10]:
def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    for y in range(3):
        for x in range(3):
            print(y*3+x+1, end=" ")
        print()
        
    state = game.initial
    while not game.is_terminal(state):
        player = state.player
        # strategies[player] calls the specific player algorithm
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose: 
            print('Player', player, 'move:', move[1]*3+move[0]+1)
            print(state)
    return state

## Step 4. Playing a Game 

We're ready to play a game. I'll set up a match between a `human player` or `random_player` (who chooses randomly from the legal moves).

Without building your own code, you can always play against random player. 

There are three outcomes of the game represeted by three numbers:

- 1 means first player wins the game
- -1 means first player loses the game
- 0 means draw

#### Human Player VS Random Player

Please run the line below to play the game:

In [11]:
play_game(TicTacToe(), dict(X=human_player, O=random_player), verbose=True)

1 2 3 
4 5 6 
7 8 9 
Player X move: 9
1 2 3
4 5 6
7 8 X

Player O move: 1
O 2 3
4 5 6
7 8 X

Player X move: 8
O 2 3
4 5 6
7 X X

Player O move: 7
O 2 3
4 5 6
O X X

Player X move: 6
O 2 3
4 5 X
O X X

Player O move: 2
O O 3
4 5 X
O X X

Player X move: 3
O O X
4 5 X
O X X



O O X
4 5 X
O X X

## Step 5. Minimax-Based Game Search Algorithms

We will define several game search algorithms. Each takes two inputs, the game we are playing and the current state of the game, and returns a a `(value, move)` pair, where `value` is the utility that the algorithm computes for the player whose turn it is to move, and `move` is the move itself.

First we define `minimax_search`, which exhaustively searches the game tree to find an optimal move (assuming both players play optimally), and `alphabeta_search`, which does the same computation, but prunes parts of the tree that could not possibly have an affect on the optimnal move.  

You need to write your code here to implement minimax and minimax with alpha-beta pruning. You should follow two pseudocode we have discussed in the activities in class:

- minimax_seach: pseudocode in Part E in activity 8
- alphabeta_search: pseudocode in Part C in activity 9

In [12]:
# For alpha-beta search, we can still use a cache, but it should be based just on the state, 
# not on whatever values alpha and beta have.
def cache1(function):
    "Like lru_cache(None), but only considers the first argument of function."
    cache = {}
    def wrapped(x, *args):
        if x not in cache:
            cache[x] = function(x, *args)
        return cache[x]
    return wrapped

In [13]:
infinity = math.inf

def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""
    # Input: game, state
    # Output: utility, move 

    player = state.player
    
    # print(game.actions(state))

    @cache
    def max_value(state):
        
        latest_action = None
        
        if game.is_terminal(state):
            return game.utility(state, player), latest_action
		
        v = -infinity
        for action in game.actions(state):
            
            if min_value(game.result(state, action))[0] > v:
                latest_action = action
                
            v = max(v, min_value(game.result(state, action))[0])
        
        return v, latest_action
        # your code goes here:
        # pass

    @cache
    def min_value(state):
        
        latest_action = None
        
        if game.is_terminal(state):
            return game.utility(state, player), latest_action
        
        v = infinity
        for action in game.actions(state):
            
            if max_value(game.result(state, action))[0] < v:
                latest_action = action
        
            v = min(v, max_value(game.result(state, action))[0])
        
        return v, latest_action
        # your code goes here:
        # pass
        
    # print(max_value(state))
    return (max_value(state))

def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning."""

    player = state.player
    
    # print(game.actions(state))

    @cache1
    def max_value(state, alpha, beta):
        
        latest_action = None
        
        if game.is_terminal(state):
            return game.utility(state, player), latest_action
		
        v = -infinity
        for action in game.actions(state):
            
            if min_value(game.result(state, action), alpha, beta)[0] > v:
                latest_action = action
                
            v = max(v, min_value(game.result(state, action), alpha, beta)[0])
            
            if v >= beta:
                return v, latest_action
            
            alpha = max(alpha, v)
        
        return v, latest_action

    @cache1
    def min_value(state, alpha, beta):
        
        latest_action = None
        
        if game.is_terminal(state):
            return game.utility(state, player), latest_action
		
        v = infinity
        for action in game.actions(state):
            
            if max_value(game.result(state, action), alpha, beta)[0] < v:
                latest_action = action
                
            v = min(v, max_value(game.result(state, action), alpha, beta)[0])
            
            if v <= alpha:
                return v, latest_action
            
            beta = min(beta, v)
        
        return v, latest_action

    return max_value(state, -infinity, +infinity)

A `player(minimax_search)` (who makes the optimal minimax move; practical for tic-tac-toe, but not for large games). The `player(minimax_search)` will never lose, but if `random_player` is lucky, it will be a tie.

If you or the random player wins over the minimax or alphabeta player, your code **must be wrong**. 

After completing your code, you can run following cells to play and verify your code. 

### Human Player VS Minimax

You never can win the game.

In [14]:
play_game(TicTacToe(), dict(X=human_player, O=AI_player(minimax_search)), verbose=True).utility

1 2 3 
4 5 6 
7 8 9 
Player X move: 9
1 2 3
4 5 6
7 8 X

Player O move: 5
1 2 3
4 O 6
7 8 X

Player X move: 8
1 2 3
4 O 6
7 X X

Player O move: 7
1 2 3
4 O 6
O X X

Player X move: 3
1 2 X
4 O 6
O X X

Player O move: 6
1 2 X
4 O O
O X X

Player X move: 4
1 2 X
X O O
O X X

Player O move: 2
1 O X
X O O
O X X

Player X move: 1
X O X
X O O
O X X



0

### Alphabeta Search VS Human Player

You never can win the game. Alphabeta moves faster then Minimax.

In [15]:
play_game(TicTacToe(), dict(X=human_player, O=AI_player(alphabeta_search)), verbose=True).utility

1 2 3 
4 5 6 
7 8 9 
Player X move: 9
1 2 3
4 5 6
7 8 X

Player O move: 5
1 2 3
4 O 6
7 8 X

Player X move: 1
X 2 3
4 O 6
7 8 X

Player O move: 4
X 2 3
O O 6
7 8 X

Player X move: 6
X 2 3
O O X
7 8 X

Player O move: 3
X 2 O
O O X
7 8 X

Player X move: 7
X 2 O
O O X
X 8 X

Player O move: 8
X 2 O
O O X
X O X

Player X move: 2
X X O
O O X
X O X



0

### Alphabeta Search VS Minimax

It will be always a draw.

In [16]:
play_game(TicTacToe(), dict(X=AI_player(alphabeta_search), O=AI_player(minimax_search)), verbose=True).utility

1 2 3 
4 5 6 
7 8 9 
Player X move: 4
1 2 3
X 5 6
7 8 9

Player O move: 6
1 2 3
X 5 O
7 8 9

Player X move: 8
1 2 3
X 5 O
7 X 9

Player O move: 1
O 2 3
X 5 O
7 X 9

Player X move: 5
O 2 3
X X O
7 X 9

Player O move: 2
O O 3
X X O
7 X 9

Player X move: 3
O O X
X X O
7 X 9

Player O move: 7
O O X
X X O
O X 9

Player X move: 9
O O X
X X O
O X X



0

## (Optional +10 points) Step 6. Count the number of nodes in the game tree.

Please copy your code for the unpruned Minimax and the Alpha-Beta Pruning Minimax below, and modify the code to count the number of nodes in the game tree for Tic-Tac-Toe. For example, in the game tree below, there are 7 nodes. 

My count for Minimax is 549,945, and for Alpha-Beta Pruning, it is 23,730. Please let me know if your count is very different from mine.

```
   root
   / |  \
  A  B  C 
 / \      \
D  E       F
```

You need to run the code below first to modify AI_player and play_game slightly to incorporate the count.

In [17]:
def AI_player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    # Only return the move, not the value of utility. 
    return lambda game, state: search_algorithm(game, state)

def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    for y in range(3):
        for x in range(3):
            print(y*3+x+1, end=" ")
        print()
        
    state = game.initial
    while not game.is_terminal(state):
        player = state.player
        # strategies[player] calls the specific player algorithm
        _, move, count = strategies[player](game, state)
        print("Count:", count)
        state = game.result(state, move)
        if verbose: 
            print('Player', player, 'move:', move[1]*3+move[0]+1)
            print(state)
    return state

In [42]:
infinity = math.inf

def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""
    # Input: game, state
    # Output: utility, move 

    player = state.player
    
    # print(game.actions(state))

    @cache
    def max_value(state):
        
        latest_action = None
        
        if game.is_terminal(state):
            return game.utility(state, player), latest_action, 1
        
        total_expanded = 1
		
        v = -infinity
        for action in game.actions(state):
            
            total_expanded += min_value(game.result(state, action))[2]
            
            if min_value(game.result(state, action))[0] > v:
                latest_action = action
                
            v = max(v, min_value(game.result(state, action))[0])
        
        return v, latest_action, total_expanded
        # your code goes here:
        # pass

    @cache
    def min_value(state):
        
        latest_action = None
        
        if game.is_terminal(state):
            return game.utility(state, player), latest_action, 1
        
        total_expanded = 1
        
        v = infinity
        for action in game.actions(state):
            
            total_expanded += max_value(game.result(state, action))[2]
            
            if max_value(game.result(state, action))[0] < v:
                latest_action = action
        
            v = min(v, max_value(game.result(state, action))[0])
        
        return v, latest_action, total_expanded
        # your code goes here:
        # pass
        
    # print(max_value(state))
    return (max_value(state))

def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning."""

    player = state.player
    
    # print(game.actions(state))

    @cache1
    def max_value(state, alpha, beta):
        
        latest_action = None
        
        if game.is_terminal(state):
            return game.utility(state, player), latest_action, 1
		
        total_expanded = 1

        v = -infinity
        for action in game.actions(state):
            
            total_expanded += min_value(game.result(state, action), alpha, beta)[2]
            
            if min_value(game.result(state, action), alpha, beta)[0] > v:
                latest_action = action
                
            v = max(v, min_value(game.result(state, action), alpha, beta)[0])
            
            if v >= beta:
                return v, latest_action, total_expanded
            
            alpha = max(alpha, v)
        
        return v, latest_action, total_expanded

    @cache1
    def min_value(state, alpha, beta):
        
        latest_action = None
        
        if game.is_terminal(state):
            return game.utility(state, player), latest_action, 1
        
        total_expanded = 1
		
        v = infinity
        for action in game.actions(state):
            
            total_expanded += max_value(game.result(state, action), alpha, beta)[2]
               
            if max_value(game.result(state, action), alpha, beta)[0] < v:
                latest_action = action
                
            v = min(v, max_value(game.result(state, action), alpha, beta)[0])
            
            if v <= alpha:
                return v, latest_action, total_expanded
            
            beta = min(beta, v)
        
        return v, latest_action, total_expanded

    return max_value(state, -infinity, +infinity)

In [45]:
print("\nMINI MAX TOTAL EXPANSION: \n")
play_game(TicTacToe(), dict(X=AI_player(minimax_search), O=AI_player(alphabeta_search))).utility

print("\nALPHA BETA TOTAL EXPANSION: \n")
play_game(TicTacToe(), dict(X=AI_player(alphabeta_search), O=AI_player(minimax_search))).utility


MINI MAX TOTAL EXPANSION: 

1 2 3 
4 5 6 
7 8 9 
Count: 549946
Count: 4534
Count: 9400
Count: 374
Count: 250
Count: 20
Count: 14
Count: 5
Count: 2

ALPHA BETA TOTAL EXPANSION: 

1 2 3 
4 5 6 
7 8 9 
Count: 23731
Count: 63905
Count: 1317
Count: 1421
Count: 142
Count: 47
Count: 13
Count: 5
Count: 2


0